In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
import keras
import pickle
import numpy as np

def resnet50_model(classes=1000, *args, **kwargs):
    # Load a model if we have saved one
    if(os.path.isfile('/content/drive/MyDrive/colab_notebook/digitalent/model/resnet_50.h5') == True):
        return keras.models.load_model('/content/drive/MyDrive/colab_notebook/digitalent/model/resnet_50.h5')
    # Create an input layer 
    input = keras.layers.Input(shape=(None, None, 3))
    # Create output layers
    output = keras.layers.ZeroPadding2D(padding=3, name='padding_conv1')(input)
    output = keras.layers.Conv2D(64, (7, 7), strides=(2, 2), use_bias=False, name='conv1')(output)
    output = keras.layers.BatchNormalization(axis=3, epsilon=1e-5, name='bn_conv1')(output)
    output = keras.layers.Activation('relu', name='conv1_relu')(output)
    output = keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='pool1')(output)
    output = conv_block(output, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    output = identity_block(output, 3, [64, 64, 256], stage=2, block='b')
    output = identity_block(output, 3, [64, 64, 256], stage=2, block='c')
    output = conv_block(output, 3, [128, 128, 512], stage=3, block='a')
    output = identity_block(output, 3, [128, 128, 512], stage=3, block='b')
    output = identity_block(output, 3, [128, 128, 512], stage=3, block='c')
    output = identity_block(output, 3, [128, 128, 512], stage=3, block='d')
    output = conv_block(output, 3, [256, 256, 1024], stage=4, block='a')
    output = identity_block(output, 3, [256, 256, 1024], stage=4, block='b')
    output = identity_block(output, 3, [256, 256, 1024], stage=4, block='c')
    output = identity_block(output, 3, [256, 256, 1024], stage=4, block='d')
    output = identity_block(output, 3, [256, 256, 1024], stage=4, block='e')
    output = identity_block(output, 3, [256, 256, 1024], stage=4, block='f')
    output = conv_block(output, 3, [512, 512, 2048], stage=5, block='a')
    output = identity_block(output, 3, [512, 512, 2048], stage=5, block='b')
    output = identity_block(output, 3, [512, 512, 2048], stage=5, block='c')
    output = keras.layers.GlobalAveragePooling2D(name='pool5')(output)
    output = keras.layers.Dense(classes, activation='softmax', name='fc1000')(output)
    # Create a model from input layer and output layers
    model = keras.models.Model(inputs=input, outputs=output, *args, **kwargs)
    # Print model
    print()
    # print(model.summary(), '\n')
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    # Return a model
    return model
# Create an identity block
def identity_block(input, kernel_size, filters, stage, block):
    
    # Variables
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    # Create layers
    output = keras.layers.Conv2D(filters1, (1, 1), kernel_initializer='he_normal', name=conv_name_base + '2a')(input)
    output = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2a')(output)
    output = keras.layers.Activation('relu')(output)
    output = keras.layers.Conv2D(filters2, kernel_size, padding='same', kernel_initializer='he_normal', name=conv_name_base + '2b')(output)
    output = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2b')(output)
    output = keras.layers.Activation('relu')(output)
    output = keras.layers.Conv2D(filters3, (1, 1), kernel_initializer='he_normal', name=conv_name_base + '2c')(output)
    output = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2c')(output)
    output = keras.layers.add([output, input])
    output = keras.layers.Activation('relu')(output)
    # Return a block
    return output
# Create a convolution block
def conv_block(input, kernel_size, filters, stage, block, strides=(2, 2)):
    # Variables
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    # Create block layers
    output = keras.layers.Conv2D(filters1, (1, 1), strides=strides, kernel_initializer='he_normal', name=conv_name_base + '2a')(input)
    output = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2a')(output)
    output = keras.layers.Activation('relu')(output)
    output = keras.layers.Conv2D(filters2, kernel_size, padding='same', kernel_initializer='he_normal', name=conv_name_base + '2b')(output)
    output = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2b')(output)
    output = keras.layers.Activation('relu')(output)
    output = keras.layers.Conv2D(filters3, (1, 1), kernel_initializer='he_normal', name=conv_name_base + '2c')(output)
    output = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2c')(output)
    shortcut = keras.layers.Conv2D(filters3, (1, 1), strides=strides, kernel_initializer='he_normal', name=conv_name_base + '1')(input)
    shortcut = keras.layers.BatchNormalization(axis=3, name=bn_name_base + '1')(shortcut)
    output = keras.layers.add([output, shortcut])
    output = keras.layers.Activation('relu')(output)
    # Return a block
    return output

In [8]:
def train():
    
    epochs = 100
    batch_size = 32
    train_samples = 26 * 12 # 26 categories with 12 images in each category
    validation_samples = 26 * 12 # 26 categories with 12 images in each category
    img_width, img_height = 32, 32
    # Get the model (26 categories)
    model = resnet50_model(26)
    # Create a data generator for training
    train_data_generator = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255, 
        shear_range=0.2, 
        zoom_range=0.2, 
        horizontal_flip=True)
    # Create a data generator for validation
    validation_data_generator = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2, 
        horizontal_flip=True)
    # Create a train generator
    train_generator = train_data_generator.flow_from_directory( 
        '/content/drive/MyDrive/colab_notebook/digitalent/dataset/Citra/', 
        target_size=(img_width, img_height), 
        batch_size=batch_size,
        color_mode='rgb',
        shuffle=True,
        class_mode='categorical')
    # Create a test generator
    validation_generator = validation_data_generator.flow_from_directory( 
        '/content/drive/MyDrive/colab_notebook/digitalent/dataset/Citra/', 
        target_size=(img_width, img_height), 
        batch_size=batch_size,
        color_mode='rgb',
        shuffle=True,
        class_mode='categorical')
    # Start training, fit the model
    hist = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=epochs,
        steps_per_epoch=train_samples // batch_size,
        validation_steps=validation_samples // batch_size,
    )
    # hist = model.fit_generator( 
    #     train_generator, 
    #     # steps_per_epoch=train_samples // batch_size, 
    #     validation_data=validation_generator, 
    #     # validation_steps=validation_samples // batch_size,
    #     epochs=epochs)
    # Save model to disk
    model.save('/content/drive/MyDrive/colab_notebook/digitalent/model/resnet_50.h5')
    print('Saved model to disk!')
    # Get labels
    labels = train_generator.class_indices
    # Invert labels
    classes = {}
    for key, value in labels.items():
        classes[value] = key.capitalize()
    # Save classes to file
    with open('/content/drive/MyDrive/colab_notebook/digitalent/model/classes.pkl', 'wb') as file:
        pickle.dump(classes, file)
    print('Saved classes to disk!')

    return hist

In [ ]:
train()

In [ ]:
input.shape
# input.reshape(1, 32, 32, 3)

(3000, 3000, 3)

In [ ]:
import keras
import cv2
import numpy as np
import pickle

model = keras.models.load_model('/content/drive/MyDrive/colab_notebook/digitalent/model/resnet_50.h5')
with open('/content/drive/MyDrive/colab_notebook/digitalent/model/classes.pkl', 'rb') as file:
        classes = pickle.load(file)
image = cv2.imread('/content/drive/MyDrive/colab_notebook/digitalent/dataset/Citra/A/wall white (3).jpg')
input = np.array(image)
input = np.array(image).reshape((1, 3000, 3000, 3)).astype('float32')/255
predictions = model.predict(input).ravel()
image = cv2.resize(image,(256,256))
#         # Print predictions
# print(predictions)
# model.summary()


In [ ]:
import cv2
import numpy as np

# img = cv2.imread('your_image.jpg')
res = cv2.resize(image, dsize=(32, 32), interpolation=cv2.INTER_CUBIC)
res.shape
# input = np.array(image)
input = np.array(res).reshape((1, 32, 32, 3)).astype('float32')/255

In [12]:
# Import libraries
import os
import cv2
import keras
import random
import pickle
import numpy as np
# Evaluate the model
def evaluate():
    # Load the model
    model = keras.models.load_model('/content/drive/MyDrive/colab_notebook/digitalent/model/resnet_50.h5')
    # Load classes
    classes = {}
    with open('/content/drive/MyDrive/colab_notebook/digitalent/model/classes.pkl', 'rb') as file:
        classes = pickle.load(file)
    # Get a list of categories
    categories = os.listdir('/content/drive/MyDrive/colab_notebook/digitalent/dataset/Citra/')
    # Get a category a random
    category = random.choice(categories)
    # Print the category
    print(category)
    # Get images in a categorydd
    images =  os.listdir('/content/drive/MyDrive/colab_notebook/digitalent/dataset/Citra/' + category)
    # Randomize images to get different images each time
    random.shuffle(images)
    # Loop images
    blocks = []
    for i, name in enumerate(images):
        # Limit the evaluation
        if i > 6:
            break;
        # Print the name
        print(name)
        # Get the image
        image = cv2.imread('/content/drive/MyDrive/colab_notebook/digitalent/dataset/Citra/' + category + '/' + name)
        # Get input reshaped and rescaled
        res = cv2.resize(image, dsize=(32, 32), interpolation=cv2.INTER_CUBIC)
        input = np.array(res).reshape((1, 32, 32, 3)).astype('float32')/255
        # Get predictions
        predictions = model.predict(input).ravel()
        # Print predictions
        print(predictions)
        # Get the class with the highest probability
        prediction = np.argmax(predictions)
        # Check if the prediction is correct
        correct = True if classes[prediction].lower() == category else False
        print(correct)
        # Draw the image and show the best prediction
        image = cv2.resize(image,(256,256))
        cv2.putText(image, '{0}: {1} %'.format(classes[prediction], str(round(predictions[prediction] * 100, 2))), (12, 22), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, '{0}: {1} %'.format(classes[prediction], str(round(predictions[prediction] * 100, 2))), (10, 20), cv2.FONT_HERSHEY_DUPLEX, 0.7, (65,105,225), 2)
        cv2.putText(image, '{0}'.format('CORRECT!' if correct else 'WRONG!'), (12, 50), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, '{0}'.format('CORRECT!' if correct else 'WRONG!'), (10, 48), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 0) if correct else (0, 0, 255), 2)
        
        # Append the image
        blocks.append(image)
        
    # Display images and predictions
    row1 = np.concatenate(blocks[0:3], axis=1)
    row2 = np.concatenate(blocks[3:6], axis=1)
    # cv2.imshow('Predictions', np.concatenate((row1, row2), axis=0))
    cv2.imwrite('predictions.jpg', np.concatenate((row1, row2), axis=0)) 
    cv2.waitKey(0)

In [15]:
evaluate()

V
wall white (1).jpg
[3.1817744e-06 1.3322223e-05 3.1187877e-04 4.4481599e-06 2.4695380e-05
 1.6751189e-06 2.2100587e-06 1.2560356e-06 4.8256913e-04 1.4364411e-05
 6.8171052e-05 1.4216435e-05 6.2408089e-06 8.2601082e-06 3.7800243e-01
 1.6630987e-05 2.1277415e-02 7.4556423e-04 5.3466065e-06 4.1149252e-07
 5.9002161e-04 5.9767705e-01 1.7294340e-06 9.4936709e-08 1.3156940e-06
 7.2554318e-04]
False
body white (3).jpg
[3.5489418e-05 1.8599548e-07 2.4020822e-04 6.9558308e-05 6.3333282e-06
 8.9108944e-08 9.4416087e-08 4.7362995e-05 4.4520055e-03 9.4518939e-05
 1.3161293e-06 3.9719874e-03 3.6898384e-06 3.7822142e-06 1.6440592e-04
 2.2079803e-05 1.9075496e-04 2.7040631e-04 8.6026339e-05 1.0965642e-06
 2.2450131e-03 9.8807436e-01 1.0067981e-05 1.3595009e-07 5.6295521e-06
 3.2638695e-06]
False
wall white (4).jpg
[8.7267317e-06 7.2380027e-07 2.8084987e-05 4.0743795e-05 1.5635458e-06
 7.5323851e-07 3.9477021e-07 5.5200842e-05 5.9560395e-04 2.5756459e-05
 1.6333928e-04 2.3773488e-05 9.2079790e-06 1.